In [ ]:
%pip install torch
%pip install clip_interrogator

In [2]:
from typing import List
from clip_interrogator import Interrogator, LabelTable

def encode_and_cache_keywords(labels:List[str], desc:str, ci: Interrogator):
    _ = LabelTable(labels, desc, ci)

In [6]:
from clip_interrogator import Config
from cliptag.cliptag import load_keywords_from_files, CIInterrogator
import torch 

if torch.cuda.is_available():
    # Use the GPU (CUDA) as the device
    device = torch.device("cuda")
    print("Using GPU:", torch.cuda.get_device_name(0))
else:
    # Use the CPU as the device
    device = torch.device("cpu")
    print("Using CPU")

model_names = [
   "ViT-L-14/openai",
   "ViT-B-32/laion2b_s34b_b79k" 
]
keywords = load_keywords_from_files("features/test")

for model_name in model_names:
  config = Config(clip_model_name=model_name, caption_model_name=None, device=device)
  ci = CIInterrogator(config)
  encode_and_cache_keywords(keywords, "keywords", ci)

Using CPU
Loading CLIP model ViT-L-14/openai...
Loading CLIP model ViT-B-32/laion2b_s34b_b79k...
